In [153]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score

In [154]:
df_results = pd.read_csv('../FilesToUploadToColab/wikipedia_predictions.csv')
df_tuples = pd.read_csv('../FilesToUploadToColab/NovelTuples.csv', sep = '\t', header = None)
df_tuples.columns = ('rel','head','tail','misc','misc2')

In [156]:
def score(tail_cond, tail_marg, head_cond, head_marg, lam = 1):
    return ((lam * (tail_cond + head_cond) - (tail_marg + head_marg)))/2
    # return lam * tail_cond + (k - 1) * head_marg - tail_marg

In [157]:
#accs = []
sils = []
lams = []
for lam in np.arange(1, 5, .05):
    df_tuples['score'] = weighted_PMI(
        df_results.tail_conditional,
        df_results.tail_marginal, 
        df_results.head_conditional, 
        df_results.head_marginal, 
        lam = lam
    )
    gm = KMeans(3).fit(df_tuples[['score']])
    sil = calinski_harabaz_score(df_tuples[['score']], gm.predict(df_tuples[['score']]))
    #acc = np.mean((gm.cluster_centers_.argmax() == df.label) == gm.predict(df_tuples[['score']]))
    sils.append(sil)
    lams.append(lam)
    #accs.append(acc)
sils = np.array(sils)
#accs = np.array(accs)
best_lam = lams[sils.argmax()]

In [175]:
best_lam = 2

In [176]:
df_tuples['score'] = weighted_PMI(
    df_results.tail_conditional,
    df_results.tail_marginal, 
    df_results.head_conditional, 
    df_results.head_marginal, 
    lam = best_lam
)
df_tuples['sent'] = df_results['sent']

In [177]:
df_top = df_tuples.sort_values('score',ascending = False)[:100]
df_top

,rel,head,tail,misc,misc2,score,sent
674,UsedFor,prayer,holy spirit,14,0.125243,6.152005,[CLS] the prayer is for the holy spirit . [SEP]
1235,IsA,atomic nucleus,atom,7,0.737449,5.874211,[CLS] an atomic nucleus is an atom . [SEP]
1948,HasPrerequisite,flood control,dam,9,0.005172,5.748349,[CLS] flood control requires a dam . [SEP]
1513,HasA,hand,index finger,30,0.724809,5.613433,[CLS] the hand has an index finger . [SEP]
1099,ReceivesAction,governor,removed from office,11,0.085226,5.607309,[CLS] the governor can be removed from office ...
1516,HasA,soil,organic matter,69,0.675219,5.603229,[CLS] the soil contains organic matter . [SEP]
2085,HasPrerequisite,sky father,earth mother,9,0.000014,5.440085,[CLS] the sky father requires the earth mother...
2177,CapableOf,gas,water vapor,20,0.181166,5.433739,[CLS] the gas can be water vapor . [SEP]
1587,HasA,limited partnership,limited liability,8,0.243739,5.048894,[CLS] the limited partnership has a limited li...
606,UsedFor,pen name,author,40,0.649584,4.915536,[CLS] the pen name is used for the author . [SEP]


In [178]:
df_top.to_csv(f'../FilesToUploadToColab/wikipedia_top_LM_lambda{np.round(best_lam, 2)}.csv')